In [1]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator
import matplotlib.pyplot as plt
import os

minmag = 4.3

project = 'summer23'
project_dir = '/hdd/Ridgecrest/' + project + '/'

tt_files_dir = project_dir + 'minM' + str(minmag) + '_seismic_traveltimes_w_stainfo/' # Loc of seismic traveltimes
p_tt_dir = tt_files_dir + 'p_tts/' # Where are the P-wave travel time calculations

gnss_tt_dir = project_dir + 'minM' + str(minmag) + '_gnss_traveltimes/' # Where to save the GNSS tts
gnss_tt_p_dir = gnss_tt_dir + 'p_tts/'
interpolation_plot_dir = gnss_tt_dir + 'interpolation_plots/'
interpplot_p_dir = interpolation_plot_dir + 'p_plots/'

if os.path.isdir(gnss_tt_dir):
    pass
else:
    os.makedirs(gnss_tt_dir)
    os.makedirs(gnss_tt_p_dir)
    os.makedirs(interpolation_plot_dir)
    os.makedirs(interpplot_p_dir)

event_catalog = np.genfromtxt(project_dir + 'minM' + str(minmag) + '_event_catalog.txt', dtype = 'U')
event_IDs = event_catalog[:,0]
GNSS_stas = np.genfromtxt(project_dir + 'GNSS_stas.txt', dtype = 'U')
GNSS_min_lon = float(min(GNSS_stas[:,0]))
GNSS_max_lon = float(max(GNSS_stas[:,0]))
GNSS_min_lat = float(min(GNSS_stas[:,1]))
GNSS_max_lat = float(max(GNSS_stas[:,1]))

counter = 0

small_test = False # Test code with smaller dataset?
resume = False

if small_test:
    event_catalog = event_catalog[23:24,:]
    p = np.where(GNSS_stas[:,2] == 'BEPK')[0]
    q = np.where(GNSS_stas[:,2] == 'P740')[0]
    r = np.where(GNSS_stas[:,2] == 'DLUZ')[0]
    
if resume:
    resume_row = 31 # Printout number it stopped at minus 1
    counter = resume_row
    event_catalog = event_catalog[resume_row:,:]

In [3]:
event_catalog[:,0]

array(['37451589', '37451541', '37451501', '37451477', '37451309',
       '37451149', '37450997', '37450989', '37450933', '37450925',
       '37450917', '37450909', '37450901', '37450893', '37450717',
       '37450613', '37450605', '37450597', '38592034', '37450541',
       '37450309', '37449909', '37449877', '39281127', '37449853',
       '37449845', '37449837', '37449757', '39273567', '37448549',
       '37447901', '39494008', '39493944', '37447701', '37447677',
       '37447637', '39490952', '37447517', '37447509', '37447501',
       '39486360', '39485504', '37447437', '37447317', '37447309',
       '37447101', '37447077', '37446725', '39462536', '37446677',
       '37446669', '37446573', '39223887', '37446517', '39223791',
       '39223527', '37446493', '37446245', '37446125', '37445997',
       '37445965', '37445829', '37445989', '39208087', '39207503',
       '37445709', '39205383', '39205151', '39204599', '39203167',
       '39202623', '37445661', '39202175', '39201767', '392015

In [ ]:
for kevent in range(len(event_catalog)):
    
    counter += 1

    event_ID = event_catalog[kevent,0]
    mag = event_catalog[kevent,5]
    
#     print('----------------------')
    print('Event ' + str(event_ID) + ' (' + str(counter) + '/' + str(len(event_IDs)) + ')')
    
    plot_event_dir = interpplot_p_dir + event_ID + '/'
    
    if os.path.isdir(plot_event_dir):
        pass
    else:
        os.makedirs(plot_event_dir)

    try:        
        
        P_array = np.load(p_tt_dir + str(event_ID) + '.npy')

        ###### Interpolate P-wave pick times ######

        P_latitude = np.asfarray(P_array[:,2])
        P_longitude = np.asfarray(P_array[:,3])
        P_travel_times = np.asfarray(P_array[:,4])

        P_interp_lat = np.linspace(GNSS_min_lat, GNSS_max_lat, num = 1000)
        P_interp_lon = np.linspace(GNSS_min_lon, GNSS_max_lon, num = 1000)

        P_interp_lon, P_interp_lat = np.meshgrid(P_interp_lon, P_interp_lat)

        P_interp = LinearNDInterpolator(list(zip(P_longitude, P_latitude)), P_travel_times)

        P_interp_tt = P_interp(P_interp_lon, P_interp_lat)

        ### Find values for GNSS stations ###

        GNSS_stas = np.genfromtxt(project_dir + 'GNSS_stas.txt', dtype = 'U')

        if small_test:  
            GNSS_stas = GNSS_stas[[p,q,r],:][:,0,:]

        GNSS_stacodes = GNSS_stas[:,2]

        GNSS_P_tts = []
        GNSS_P_stas = []
        P_event_IDs = []
        P_dists_to_interp_pt_deg = []

        for kGsta in range(len(GNSS_stacodes)):

            P_dists = []
            lon_list = []
            lat_list = []

            GNSS_stacode = GNSS_stas[kGsta,2]
#             print('--- ' + str(GNSS_stacode) + ' ---')
            GNSS_lon = float(GNSS_stas[kGsta,0])
            GNSS_lat = float(GNSS_stas[kGsta,1])

#             print('GNSS station location: (' + str(GNSS_lat) + ', ' + str(GNSS_lon) + ')')

            for i_lon in P_interp_lon[0]:
                # print(i_lon)
                for i_lat in P_interp_lat[:,0]:
                    # print(i_lat)
                    dist = np.sqrt((GNSS_lon - i_lon)**2 + (GNSS_lat - i_lat)**2)

                    P_dists.append(dist)
                    lon_list.append(i_lon)
                    lat_list.append(i_lat)

            P_dists = np.array(P_dists)
            lon_array = np.array(lon_list)
            lat_array = np.array(lat_list)   
            i = np.argmin(P_dists)
            P_dist_to_interp_pt_deg = [P_dists[i]]
#             print('Distance (km) to closest interpolated point: ' + str(round(P_dists[i] * 111, 1)))
#             print('Interpolated point location: (' + str(lat_array[i]) + ', ' + str(lon_array[i]) + ')')

            a = np.where(P_interp_lon[0] == lon_array[i])[0]
            b = np.where(P_interp_lat[:,0] == lat_array[i])[0]
            GNSS_time = P_interp_tt[b,a][0]
#             print('Interpolated GNSS travel time: ' + str(round(GNSS_time, 2)))

            GNSS_P_tts.append(GNSS_time)
            GNSS_P_stas.append(GNSS_stacode)
            P_event_IDs.append(event_ID)
            
            

            plt.pcolormesh(P_interp_lon, P_interp_lat, P_interp_tt, shading = 'auto')
            plt.plot(P_longitude, P_latitude, '.k', label = 'Seismic stations')
            plt.plot(GNSS_lon, GNSS_lat, '.', color = 'red', label = 'GNSS station ' + GNSS_stacode)
            plt.plot()
            plt.legend()
            plt.colorbar(label = 'Travel time (s)')
            plt.xlabel('Longitude')
            plt.ylabel('Latitude')
            plt.title('P-wave travel times: event ID ' + str(event_ID) + ' (magnitude ' + str(mag) + ')')
#             plt.show()
            plt.savefig(plot_event_dir + str(GNSS_stacode) + '.png', format = 'PNG')
            plt.close()

#         print(GNSS_P_tts)    
#         print(GNSS_P_stas)

        GNSS_P_array = np.column_stack((np.array(P_event_IDs), np.array(GNSS_P_stas), np.array(GNSS_P_tts)))

        np.save(gnss_tt_p_dir + str(event_ID) + '.npy', GNSS_P_array)
        
    except:
        
        print('Error for this event')
        nan_P_array = np.column_stack(('nan', 'nan', 'nan'))
        np.save(gnss_tt_p_dir + str(event_ID) + '.npy', nan_P_array)